In [1]:
from torchvision import models, transforms
import torch
from dataset import Imagenette, get_base_transform
from trainer import Trainer

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: "{device}"')

device: "cuda:0"


In [2]:
base_transform = get_base_transform(128)

train_transform = transforms.Compose([
    transforms.RandomRotation(degrees=180),
    transforms.RandomHorizontalFlip(),
    base_transform
])

In [3]:
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 10)

/home/nils/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/nils/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
root_dir = 'imagenette2'
training_set = Imagenette(root_dir, 'train', transform=train_transform)
validation_set = Imagenette(root_dir, 'val', transform=base_transform)

batch_size = 2**6
training_loader = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True, num_workers=2)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size, shuffle=False, num_workers=2)

print(len(training_set), len(validation_set))

9469 3925


In [5]:
trainer = Trainer(
    loss_func=torch.nn.CrossEntropyLoss(),
    optimizer=torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9),
    device=device,
    summary_writer=writer
)

trainer.fit(
    model, 
    training_loader,
    validation_loader,
    num_epochs=5,
    evaluate_each_epoch=True
)

KeyboardInterrupt: 